In [ ]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-u_a7s_ir
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-u_a7s_ir
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [ ]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [ ]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"

#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl
#include <limits> // for +infinity value
#include <sstream>

#include "cuda_common.h"

    namespace csc485b {
    namespace a1 {
        namespace gpu {

            /**
             * The CPU baseline benefits from warm caches because the data was generated on
             * the CPU. Run the data through the GPU once with some arbitrary logic to
             * ensure that the GPU cache is warm too and the comparison is more fair.
             */
            __global__
                void warm_the_gpu(element_t* data, std::size_t invert_at_pos, std::size_t num_elements)
            {
                int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

                // We know this will never be true, because of the data generator logic,
                // but I doubt that the compiler will figure it out. Thus every element
                // should be read, but none of them should be modified.
                if (th_id < num_elements && data[th_id] > num_elements * 100)
                {
                    ++data[th_id]; // should not be possible.
                }
            }

            // Used this StackOverflow post for algorithm to find next power of 2.
            // https://stackoverflow.com/questions/364985/algorithm-for-finding-the-smallest-power-of-two-thats-greater-or-equal-to-a-giv
            // Commented out since we do not have to worry about input not being power of two.
            //std::size_t next_power_of_two(element_t x) {
            //    --x;
            //    x |= x >> 1;
            //    x |= x >> 2;
            //    x |= x >> 4;
            //    x |= x >> 8;
            //    x |= x >> 16;
            //    return x + 1;
            //}

            /**
             * Your solution. Should match the CPU output.
             */
            __global__
                void opposing_sort(element_t* data, std::size_t invert_at_pos, std::size_t num_elements)
            {
                element_t const th_id = blockIdx.x * blockDim.x + threadIdx.x;
                extern __shared__ element_t s_data[];

                if (th_id < num_elements)
                {
                    // Load the data from global memory to shared memory
                    // s_data[threadIdx.x] = data[th_id];

                    // Perform Bitonic Sort but DIFFERENT
                    for (std::size_t stage = 2; stage <= num_elements; stage <<= 1) {
                        for (std::size_t step = stage >> 1; step > 0; step >>= 1) {
                            // Determine the index of the element to compare with
                            std::size_t partner = th_id ^ step;

                            if (partner > th_id && partner < num_elements) {
                                // Ascending if th_id and partner are in the same stage group (because stage shifts by 1 bit each time, this works nicely)
                                bool ascending = (th_id & stage) == 0;

                                // Compare and swap based on direction
                                if (ascending) {
                                    if (data[th_id] > data[partner]) {
                                        element_t temp = data[th_id];
                                        data[th_id] = data[partner];
                                        data[partner] = temp;
                                    }
                                }
                                else {
                                    // Descending order
                                    if (data[th_id] < data[partner]) {
                                        element_t temp = data[th_id];
                                        data[th_id] = data[partner];
                                        data[partner] = temp;
                                    }
                                }
                            }
                            __syncthreads();
                        }
                    }

                    // Copy the sorted data back to global memory for this thread
                    // data[th_id] = s_data[threadIdx.x];

                    // Reverses array at invert position onwards.
                    if (th_id >= invert_at_pos) {
                        s_data[num_elements - 1 - th_id] = data[th_id];
                        __syncthreads();
                        int outOffset = blockDim.x * (blockIdx.x);
                        int out = outOffset + th_id;
                        data[out] = s_data[th_id - invert_at_pos];
                    }
                }
            }

            __global__
            void opposing_sort_step( element_t * data, std::size_t num_elements, int j, int k, bool direction )
            {
                int const th_id = blockIdx.x * blockDim.x + threadIdx.x;
                // Determine the index of the element to compare with
                int partner = th_id ^ j;

                if (partner > th_id && partner < num_elements) {
                    // Ascending if th_id and partner are in the same stage group (because stage shifts by 1 bit each time, this works nicely)
                    // If direction is false, the boolean value for ascending is flipped.
                    bool ascending = ((th_id & k) == 0) ^ !direction;

                    // Compare and swap based on direction
                    if (ascending) {
                        if (data[th_id] > data[partner]) {
                            element_t temp = data[th_id];
                            data[th_id] = data[partner];
                            data[partner] = temp;
                        }
                    } else {
                        if (data[th_id] < data[partner]) {
                            element_t temp = data[th_id];
                            data[th_id] = data[partner];
                            data[partner] = temp;
                        }
                    }
                }
            }


            /**
             * Performs all the logic of allocating device vectors and copying host/input
             * vectors to the device. Times the opposing_sort() kernel with wall time,
             * but excludes set up and tear down costs such as mallocs, frees, and memcpies.
             */
            void run_gpu_soln(std::vector< element_t > data, std::size_t switch_at, std::size_t n)
            {

                // Check if input is not a power of 2, and then pad input to a power of 2.
                // Code has been commented out as test cases will only be powers of 2.
                // if (n & (n-1)) {
                //    std::size_t next_power_of_2 = next_power_of_two(n);
                //    for (std::size_t i = 0; i < (next_power_of_2 - n); ++i) {
                //        data.push_back(std::numeric_limits<element_t>::max()-1);
                //    }
                //    n = next_power_of_2;
                //}
                std::size_t const threads_per_block = 1024;
                std::size_t const num_blocks = (n + threads_per_block - 1) / threads_per_block;

                // Allocate arrays on the device/GPU
                element_t* d_data;
                cudaMalloc((void**)&d_data, sizeof(element_t) * n);
                CHECK_ERROR("Allocating input array on device");

                // Copy the input from the host to the device/GPU
                cudaMemcpy(d_data, data.data(), sizeof(element_t) * n, cudaMemcpyHostToDevice);
                CHECK_ERROR("Copying input array to device");

                // Warm the cache on the GPU for a more fair comparison
                warm_the_gpu << < num_blocks, threads_per_block >> > (d_data, switch_at, n);

                // Time the execution of the kernel that you implemented
                auto const kernel_start = std::chrono::high_resolution_clock::now();
                auto const smem_size = threads_per_block * sizeof(element_t);

                // Run with a single thread block if input size allows
                if (n <= 1024) {
                    opposing_sort << < num_blocks, threads_per_block, smem_size >> > (d_data, switch_at, n);
                }
                else {
                    // Otherwise do inter-block synchronization in CPU code
                    int j, k;
                    // Major step
                    for (k = 2; k <= n; k <<= 1) {
                        // Minor step
                        for(j = k >> 1; j > 0; j >>= 1) {
                        opposing_sort_step<<< num_blocks, threads_per_block>>>( d_data, n, j, k, true);
                        }
                    }
                    // Re-sort last quarter in reverse order
                    for (k = 2; k <= n; k <<= 1) {
                        // Minor step
                        for(j = k >> 1; j > 0; j >>= 1) {
                        opposing_sort_step<<< num_blocks, threads_per_block>>>( d_data + switch_at, n - switch_at, j, k, false);
                        }
                    }
                }
                // Remove positive infinities from padded input if original input was not a power of two.
                // Commented out since input is guaranteed to be a power of two in the test cases.
                 //std::vector<element_t> result;
                 //for (std::size_t i = 0; i < n; ++i) {
                 //    if (data[i] != std::numeric_limits<element_t>::max() - 1) {
                 //        result.push_back(data[i]);
                 //    }
                 //}

                auto const kernel_end = std::chrono::high_resolution_clock::now();
                //for (auto const x : result) std::cout << x << " "; std::cout << std::endl;
                CHECK_ERROR("Executing kernel on device");

                // After the timer ends, copy the result back, free the device vector,
                // and echo out the timings and the results.
                cudaMemcpy(data.data(), d_data, sizeof(element_t) * n, cudaMemcpyDeviceToHost);
                CHECK_ERROR("Transferring result back to host");
                cudaFree(d_data);
                CHECK_ERROR("Freeing device memory");

                std::cout << "GPU Solution time: "
                    << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
                    << " ns" << std::endl;

                //for (auto const x : data) std::cout << x << " "; std::cout << std::endl;
                std::ostringstream buffer;

                // Simulate writing many strings to the buffer
                for (std::size_t i = 0; i < n; ++i) {
                    buffer << data[i] << " ";
                }
                buffer << '\n';

                // Print the entire buffer content to console with a single std::cout call
                std::cout << buffer.str();
            }

        } // namespace gpu
    } // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n = 8;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );
    csc485b::a1::cpu::run_cpu_baseline( data, switch_at, n );
    csc485b::a1::gpu::run_gpu_soln( data, switch_at, n );

    return EXIT_SUCCESS;
}

In [ ]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

CPU Baseline time: 473 ns
0 0 3 6 10 13 15 13 
GPU Solution time: 13806 ns
0 0 3 6 10 13 15 13 

